In [ ]:
import redis
import cherrypy
import json

# Redis connection setup
redis_host = 'redis-19881.c268.eu-west-1-2.ec2.cloud.redislabs.com'
redis_port = 19881
redis_password = 'Z3uevMG7GPbr5g66RbiGFWrQhsgKLdfO'
redis_timeseries_key = 'listening_to'

redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password)
try:
    is_connected = redis_client.ping()
    print('Redis client is connected ', is_connected)
except Exception as ex:
    print(ex)

class SensorData:
    @cherrypy.expose
    @cherrypy.tools.json_out()
    def sensors(self):
        # Retrieve all sensor data from Redis TimeSeries
        print("Received request on sensors endpoint")
        data = redis_client.execute_command('TS.RANGE', 'listening_to', '-', '+')
        response = {"values": []}
        for d in data:
            response["values"].append({"timestamp": d[0], "value": int(d[1])})

        # Return the data as a JSON response
        print(response)
        return json.dumps(response)

    @cherrypy.expose
    @cherrypy.tools.json_out()
    def sensor(self, id: str):
        # Retrieve sensor data for a specific ID from Redis TimeSeries
        # Retrieve all sensor data from Redis TimeSeries
        print("Received request on sensors endpoint")
        data = redis_client.execute_command('TS.RANGE', 'listening_to_' + id, '-', '+')
        response = {"values": []}
        for d in data:
            response["values"].append({"timestamp": d[0], "value": int(d[1])})

        # Return the data as a JSON response
        print(response)
        return json.dumps(response)

cherrypy.tree.mount(SensorData(), '/')

if __name__ == '__main__':
    cherrypy.config.update({
        'server.socket_host': '0.0.0.0',
        'server.socket_port': 8080
    })
    cherrypy.engine.start()

[02/Jul/2023:18:02:00] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[02/Jul/2023:18:02:00] ENGINE Started monitor thread 'Autoreloader'.
[02/Jul/2023:18:02:00] ENGINE Serving on http://0.0.0.0:8080
[02/Jul/2023:18:02:00] ENGINE Bus STARTED
Redis client is connected  True


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e190a0d-e0c4-4d69-ad9e-7a1f03e6c992' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>